In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from typing import Iterable
import time

import libs.conv as conv
import libs.max_pool as max_pool

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils import data

dev = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(dev)
print(device)

cpu


In [2]:
bsz, channels, depth, height, width = 4, 1, 3, 224, 224
input_size = (bsz, channels, depth, height, width)
padding = (2, 2, 2, 2, 2, 2)
kernel_size = (3, 3, 3)
stride = (1, 1, 1)
X = torch.rand(input_size)
print(X.shape)

torch.Size([4, 1, 3, 224, 224])


In [3]:
def compare_times(layer1, layer2, n_trials):
    times1, times2 = [], []
    for _ in range(n_trials):
        start = time.time()
        out = layer1(X)
        times1.append(time.time() - start)

        start = time.time()
        out = layer2(X)
        times2.append(time.time() - start)
    return times1, times2

In [4]:
# custom_conv3d = conv.CustomConv3d(in_channels=3, out_channels=5, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
# control_conv3d = nn.Conv3d(in_channels=3, out_channels=5, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
# custom_times, control_times = [], []

# n_trials = 100
# custom_times, control_times = compare_times(custom_conv3d, control_conv3d, n_trials)
# print(f'mean\ncustom: {np.mean(custom_times)}\ncontrol: {np.mean(control_times)}\nstd\ncustom: {np.std(custom_times)}\ncontrol: {np.std(control_times)}')

In [5]:
class Custom3dNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = conv.CustomConv3d(in_channels=1, out_channels=96, kernel_size=(3, 11, 11), stride=(1, 5, 5), padding=(1, 5, 5))
        self.conv2 = conv.CustomConv3d(in_channels=96, out_channels=256, kernel_size=(3, 9, 9), stride=(1, 5, 5), padding=(1, 5, 5))
        self.conv3 = conv.CustomConv3d(in_channels=256, out_channels=512, kernel_size=(1, 7, 7), stride=(1, 3, 3), padding=(1, 3, 3))
        self.conv4 = conv.CustomConv3d(in_channels=512, out_channels=512, kernel_size=(1, 5, 5), stride=(1, 3, 3), padding=(1, 3, 3))
        self.conv5 = conv.CustomConv3d(in_channels=512, out_channels=256, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.conv6 = conv.CustomConv3d(in_channels=256, out_channels=96, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))

        self.fc1 = nn.Linear(2640000, 4096)
        self.fc2 = nn.Linear(4096, 1000)
        self.fc3 = nn.Linear(1000, 10)
        
        self.pool = max_pool.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1))
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm3d(96)
        self.bn2 = nn.BatchNorm3d(256)
        self.bn3 = nn.BatchNorm3d(512)
        self.bn4 = nn.BatchNorm3d(512)
        self.bn5 = nn.BatchNorm3d(256)
        self.bn6 = nn.BatchNorm3d(96)
        self.flatten = nn.Flatten()
    
    def forward(self, X):
        out = self.bn1(self.relu(self.conv1(X)))
        out = self.bn2(self.relu(self.conv2(out)))
        out = self.pool(out)
        out = self.bn3(self.relu(self.conv3(out)))
        out = self.bn4(self.relu(self.conv4(out)))
        out = self.pool(out)
        out = self.bn5(self.relu(self.conv5(out)))
        out = self.bn6(self.relu(self.conv6(out)))
        out = self.pool(out)
        out = self.flatten(out)
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [6]:
class Traditional3dNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=96, kernel_size=(3, 11, 11), stride=(1, 5, 5), padding=(1, 5, 5))
        self.conv2 = nn.Conv3d(in_channels=96, out_channels=256, kernel_size=(3, 9, 9), stride=(1, 5, 5), padding=(1, 5, 5))
        self.conv3 = nn.Conv3d(in_channels=256, out_channels=512, kernel_size=(1, 7, 7), stride=(1, 3, 3), padding=(1, 3, 3))
        self.conv4 = nn.Conv3d(in_channels=512, out_channels=512, kernel_size=(1, 5, 5), stride=(1, 3, 3), padding=(1, 3, 3))
        self.conv5 = nn.Conv3d(in_channels=512, out_channels=256, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.conv6 = nn.Conv3d(in_channels=256, out_channels=96, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))

        # self.fc1 = nn.Linear(2640000, 4096)
        self.fc2 = nn.Linear(4096, 1000)
        self.fc3 = nn.Linear(1000, 10)
        
        self.pool = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1))
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm3d(96)
        self.bn2 = nn.BatchNorm3d(256)
        self.bn3 = nn.BatchNorm3d(512)
        self.bn4 = nn.BatchNorm3d(512)
        self.bn5 = nn.BatchNorm3d(256)
        self.bn6 = nn.BatchNorm3d(96)
        self.flatten = nn.Flatten()
    
    def forward(self, X):
        out = self.bn1(self.relu(self.conv1(X)))
        out = self.bn2(self.relu(self.conv2(out)))
        out = self.pool(out)
        out = self.bn3(self.relu(self.conv3(out)))
        out = self.bn4(self.relu(self.conv4(out)))
        out = self.pool(out)
        out = self.bn5(self.relu(self.conv5(out)))
        out = self.bn6(self.relu(self.conv6(out)))
        out = self.pool(out)
        # out = self.flatten(out)
        # out = self.relu(self.fc1(out))
        # out = self.relu(self.fc2(out))
        # out = self.fc3(out)
        return out

In [7]:
def softmax(X):
    out = torch.exp(X)
    return out / torch.sum(out, axis=-1, keepdim=True)

In [8]:
traditional_net = Traditional3dNetwork()

traditional_out = traditional_net(X)
# traditional_out = softmax(traditional_out)

print(traditional_out.shape, traditional_out.min(), traditional_out.max())

RuntimeError: Given input size: (512x7x2x2). Calculated output size: (512x7x0x0). Output size is too small